In [1]:
sh_database = "YOUR_SCHEMA"
sh_table = "TABLE_WITH_YOUR_SIGNAL_BEFORE_ESG"

as_of_date_database = "YOUR_SCHEMA"
as_of_date_table = "daily_dates_with_weekends"

esg_analytic_database = "YOUR_SCHEMA"
esg_analytic_table = "esgpitanalyticvalue_fixed"

esg_value_database = "YOUR_SCHEMA"
esg_value_table = "esgpitvalues_fixed"

esg_score_database = "YOUR_SCHEMA"
esg_score_table = "esgpitscores_fixed"

analytics_factors = ['TotalCO2EquivalentsEmissionsToRevenues', 'VocEmissionsToRevenues', 'TotalWasteToRevenues', 'TotalHazardousWasteToRevenues', 'WaterPollutantEmissionsToRevenues', 'InjuriesToMillionHours', 'TotalRenewableEnergyToEnergyUse']
value_factors = ['VoluntaryTurnoverOfEmployees', 'WomenManagers', 'AverageTrainingHours', 'CustomerSatisfaction']
score_factors = ['ESGManagementScore', 'ESGShareholdersScore']

esg_as_of_dates_database = "YOUR_SCHEMA"
esg_as_of_dates_table = "esg_factors_for_as_of_dates_keyvalue"
esg_as_of_dates_table_s3_dir = "s3://YOUR_BUCKET/data/repo/esgsh_esg_pit/"
esg_as_of_dates_table_s3_fullpath = esg_as_of_dates_table_s3_dir + esg_as_of_dates_table

pyathena_staging = "s3:/YOUR_BUCKET/data/repo/esgsh_esg_pit/pyathena/"

# ===================================
# DON'T CHANGE ANYTHING BELOW HERE
# ===================================

In [2]:
!pip install pyathena 

In [3]:
import pyathena 

In [4]:
# Make the lists into friendly strings for SQL
analytics_str = ', '.join(f"'{w}'" for w in analytics_factors)
values_str = ', '.join(f"'{w}'" for w in value_factors)
scores_str = ', '.join(f"'{w}'" for w in score_factors)

In [5]:
esg_holdingdate_sql = f"""
    CREATE TABLE "{esg_as_of_dates_database}"."{esg_as_of_dates_table}"
    WITH (format = 'Parquet', parquet_compression = 'SNAPPY', external_location ='{esg_as_of_dates_table_s3_fullpath}')
    AS
    WITH sh_universe
    AS
    (
        -- Focus only on securities in Smart Holdings universe
        SELECT orgpermid,
        MIN(as_of_date) AS first_date,
        MAX(as_of_date) AS last_date
        FROM "{sh_database}"."{sh_table}"
        GROUP BY orgpermid
    ),
    most_recent_period_a
    AS
    (
        -- For each Company/AsOfDate/DataItem, figure out the most recently reported Period
        -- But stop lookback at 2 years ago because that's too old
        SELECT u.orgpermid,
        d.as_of_date,
        e.itemcode,
        MAX(e.periodenddate) AS periodenddate
        FROM sh_universe u
        INNER JOIN "{as_of_date_database}"."{as_of_date_table}" d
        ON d.as_of_date BETWEEN u.first_date AND u.last_date
        INNER JOIN "{esg_analytic_database}"."{esg_analytic_table}" e
        ON u.orgpermid = e.organizationid
        AND d.as_of_date >= e.effectivefromdate
        AND e.periodenddate > DATE_ADD('year',-2, d.as_of_date) 
        AND e.itemcode IN ({analytics_str})
        GROUP BY u.orgpermid, d.as_of_date, e.itemcode
    ),
    most_recent_period_v
    AS
    (
        -- For each Company/AsOfDate/DataItem, figure out the most recently reported Period
        -- But stop lookback at 2 years ago because that's too old
        SELECT u.orgpermid,
        d.as_of_date,
        e.itemcode,
        MAX(e.periodenddate) AS periodenddate
        FROM sh_universe u
        INNER JOIN "{as_of_date_database}"."{as_of_date_table}" d
        ON d.as_of_date BETWEEN u.first_date AND u.last_date
        INNER JOIN "{esg_value_database}"."{esg_value_table}" e
        ON u.orgpermid = e.organizationid
        AND d.as_of_date >= e.effectivefromdate
        AND e.periodenddate > DATE_ADD('year',-2, d.as_of_date) 
        AND e.itemcode IN ({values_str})
        GROUP BY u.orgpermid, d.as_of_date, e.itemcode
    ),
    most_recent_period_s
    AS
    (
        -- For each Company/AsOfDate/DataItem, figure out the most recently reported Period
        -- But stop lookback at 2 years ago because that's too old
        SELECT u.orgpermid,
        d.as_of_date,
        e.itemcode,
        MAX(e.periodenddate) AS periodenddate
        FROM sh_universe u
        INNER JOIN "{as_of_date_database}"."{as_of_date_table}" d
        ON d.as_of_date BETWEEN u.first_date AND u.last_date
        INNER JOIN "{esg_score_database}"."{esg_score_table}" e
        ON u.orgpermid = e.organizationid
        AND d.as_of_date >= e.effectivefromdate
        AND e.periodenddate > DATE_ADD('year',-2, d.as_of_date) 
        AND e.itemcode IN ({scores_str})
        GROUP BY u.orgpermid, d.as_of_date, e.itemcode
    )
    -- Pulling in all ESG Analytics of interest
    SELECT d.orgpermid,
    d.as_of_date,
    d.itemcode,
    d.periodenddate,
    MAX(e.effectivefromdate) AS effectivefromdate,
    MAX_BY(e.esganalyticvalue, e.periodenddate) AS esg_value
    FROM most_recent_period_a d
    INNER JOIN "{esg_analytic_database}"."{esg_analytic_table}" e
    ON d.orgpermid = e.organizationid
    AND d.periodenddate = e.periodenddate
    AND d.as_of_date >= e.effectivefromdate
    AND d.itemcode=e.itemcode
    GROUP BY d.orgpermid, d.as_of_date, d.itemcode, d.periodenddate
    UNION
    -- Pulling in all ESG Values of interest
    SELECT d.orgpermid,
    d.as_of_date,
    d.itemcode,
    d.periodenddate,
    MAX(e.effectivefromdate) AS effectivefromdate,
    MAX_BY(e.esgvalue, e.periodenddate) AS esg_value
    FROM most_recent_period_v d
    INNER JOIN "{esg_value_database}"."{esg_value_table}" e
    ON d.orgpermid = e.organizationid
    AND d.periodenddate = e.periodenddate
    AND d.as_of_date >= e.effectivefromdate
    AND d.itemcode=e.itemcode
    GROUP BY d.orgpermid, d.as_of_date, d.itemcode, d.periodenddate
    UNION
    -- Pulling in all ESG Scores of interest
    SELECT d.orgpermid,
    d.as_of_date,
    d.itemcode,
    d.periodenddate,
    MAX(e.effectivefromdate) AS effectivefromdate,
    MAX_BY(e.esganalyticvalue, e.periodenddate) AS esg_value
    FROM most_recent_period_s d
    INNER JOIN "{esg_score_database}"."{esg_score_table}" e
    ON d.orgpermid = e.organizationid
    AND d.periodenddate = e.periodenddate
    AND d.as_of_date >= e.effectivefromdate
    AND d.itemcode=e.itemcode
    GROUP BY d.orgpermid, d.as_of_date, d.itemcode, d.periodenddate
    """

### Here's what the SQL looks like with all the variables in it

In [ ]:
print(esg_holdingdate_sql) 

### Now connect to Athena and run the query

In [7]:
athena_cur = pyathena.connect(s3_staging_dir=pyathena_staging).cursor() 

In [8]:
athena_cur.execute(esg_holdingdate_sql)
print(athena_cur.query_id) 

897a74f9-7fc9-4621-8a7c-ab2249dca62c
